<a href="https://colab.research.google.com/github/danielmachlab/imdb_review_classifier/blob/development/imdb_review_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB Review Classifier

### Dev Steps
1. Import Dataset from keras/tf
2. View Sentences
3. Translate encoded reviews 
4. Create Model
5. Train Model
6. Test Model
7. Tweak Hyper Parameters


In [0]:
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb

### View Data

In [0]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

In [6]:
word_index = imdb.get_word_index()
# shift all numbers by three to make room for placeholders
word_index = {k: (v+3) for k, v in word_index.items()} 

# add placeholders to the word_index dictionary
word_index.update({"<PAD>": 0, "<START>": 1, "<UNK>": 2, "<UNUSED>": 3})
list(word_index.items())[-10:]

1646592/1641221 [==============================] - 0s 0us/step


[('hardboiled', 52008),
 ("voorhees'", 88586),
 ('35mm', 16818),
 ("'l'", 88587),
 ('paget', 18512),
 ('expands', 20600),
 ('<PAD>', 0),
 ('<START>', 1),
 ('<UNK>', 2),
 ('<UNUSED>', 3)]

In [7]:
# view the first review
print('sentence', x_train[0])
print('label', y_train[1])

sentence [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
label 0


### Translate Encoded Reviews

In [0]:
flipped_word_index = dict([ (v,k) for (k, v) in word_index.items() ])

In [0]:
def translate_review(review):
  return " ".join( [flipped_word_index.get(word, '?') for word in review] )

In [10]:
translate_review(x_train[0])

"<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wh

In [0]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, value=word_index["<PAD>"], padding="post", maxlen=250)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, value=word_index["<PAD>"], padding="post", maxlen=250)

In [11]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10000, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, w

In [14]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=40, batch_size=512, validation_split=0.3)

Train on 17500 samples, validate on 7500 samples
Epoch 1/40
17500/17500 [==============================] - 1s 66us/sample - loss: 0.6920 - acc: 0.5377 - val_loss: 0.6895 - val_acc: 0.6972
Epoch 2/40
17500/17500 [==============================] - 1s 43us/sample - loss: 0.6850 - acc: 0.7179 - val_loss: 0.6789 - val_acc: 0.7131
Epoch 3/40
17500/17500 [==============================] - 1s 43us/sample - loss: 0.6676 - acc: 0.7508 - val_loss: 0.6536 - val_acc: 0.7563
Epoch 4/40
17500/17500 [==============================] - 1s 43us/sample - loss: 0.6344 - acc: 0.7813 - val_loss: 0.6142 - val_acc: 0.7796
Epoch 5/40
17500/17500 [==============================] - 1s 45us/sample - loss: 0.5869 - acc: 0.8083 - val_loss: 0.5638 - val_acc: 0.8001
Epoch 6/40
17500/17500 [==============================] - 1s 44us/sample - loss: 0.5308 - acc: 0.8341 - val_loss: 0.5112 - val_acc: 0.8176
Epoch 7/40
17500/17500 [==============================] - 1s 43us/sample - loss: 0.4737 - acc: 0.8508 - val_loss: 0.4